In [5]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [15]:
def grayscale(img):
    grayimg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)#gray scale conversion
    return grayimg

def gaussian_blur(img, kernel_size):
    blur_grayimg = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    return blur_grayimg

def canny(img, low_threshold, high_threshold):
    edges = cv2.Canny(img, low_threshold, high_threshold)
    return edges

def region_of_interest(img, vertices):
    mask= np.zeros_like(img) 
    poly = cv2.fillPoly(mask, vertices, 255)
    masked_edges = cv2.bitwise_and(img, mask)
    return masked_edges

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                                min_line_len, max_line_gap)
    return lines
    

In [13]:
def draw_lines(img, lines):
    global x_leftprev
    global y_leftprev
    global x_rightprev
    global y_rightprev
    x_left=[]
    y_left=[]
    x_right=[]
    y_right=[]

    for line in lines:
        for x1,y1,x2,y2 in line:
            if (x1<500) and (x2<500):
                x_left.append(x1)
                x_left.append(x2)
                y_left.append(y1)
                y_left.append(y2)
            else:
                x_right.append(x1)
                x_right.append(x2)
                y_right.append(y1)
                y_right.append(y2)
    if(x_left!=[] and y_left!=[]):
        
        x_leftprev=x_left
        y_leftprev=y_left
        left_fit = np.polyfit(x_left,y_left,1)
        #left lane min and max coordinates
        min_xleft = min(x_left)
        max_xleft = max(x_left)
    else:
        left_fit = np.polyfit(x_leftprev,y_leftprev,1)
        #left lane min and max coordinates
        min_xleft = min(x_leftprev)
        max_xleft = max(x_leftprev)
        
    if (x_right!=[] and y_right!=[]):
        right_fit = np.polyfit(x_right,y_right,1)
        x_rightprev = x_right
        y_rightprev=y_right     
        #right lane min and max coordinates
        min_xryt = min(x_right)
        max_xryt = max(x_right)
    else:
        right_fit= np.polyfit(x_rightprev,y_rightprev,1)      
        #right lane min and max coordinates
        min_xryt = min(x_rightprev)
        max_xryt = max(x_rightprev)
        
    #coeficients of the left and right lanes
    m_left = left_fit[0]
    b_left =left_fit[1]
    m_right = right_fit[0]
    b_right = right_fit[1]
    
    #left lane
    
    min_yleft = m_left*min_xleft+b_left
    max_yleft = m_left*max_xleft+b_left
    #print("xminleft,yminleft,xmaxleft,ymaxleft")
    #print(int(min_xleft),int(min_yleft),int(max_xleft),int(max_yleft))
    cv2.line(img,(int(min_xleft),int(min_yleft)),(int(max_xleft),int(max_yleft)),[255,0,0],thickness=15)
    
    #right lane
    
    min_yryt = m_right*min_xryt+b_right
    max_yryt = m_right*max_xryt+b_right
    #print("xminright,yminright,xmaxright,ymaxright")
    #print(int(min_xryt),int(min_yryt),int(max_xryt),int(max_yryt))
    cv2.line(img,(int(min_xryt),int(min_yryt)),(int(max_xryt),int(max_yryt)),[255,0,0],thickness=15)
    
    return img
    
                

In [20]:
def process_image(image):
    grayimg = grayscale(image)
    #grayimg = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)#gray scale conversion
    kernel_size=7
    low_threshold = 50
    high_threshold = 150
    blur_grayimg=gaussian_blur(grayimg,kernel_size)
    #blur_grayimg = cv2.GaussianBlur(grayimg, (kernel_size, kernel_size), 0)
    edges = canny(blur_grayimg, low_threshold, high_threshold)
    #edges = cv2.Canny(blur_grayimg, low_threshold, high_threshold)
    #plt.imshow(edges, cmap='gray')
    img_shape = image.shape
    #print(img_shape)
    #mask= np.zeros_like(edges)
    ROI_vertices = np.array([[(0,img_shape[0]),(490, 310), (img_shape[1],img_shape[0])]], dtype=np.int32) 
    masked_edges = region_of_interest(edges,ROI_vertices)
    #poly = cv2.fillPoly(mask, ROI_vertices, 255)
    #plt.imshow(poly,cmap='gray')
    #masked_edges = cv2.bitwise_and(edges, mask)
    #plt.imshow(masked_edges, cmap='gray')
    rho = 2
    theta = np.pi/180 
    threshold = 10     
    min_line_length = 25 
    max_line_gap = 45
    line_image = np.copy(image)*0 
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    #lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
    #                            min_line_length, max_line_gap)
    #print(lines)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    
    
    line_output = draw_lines(line_image,lines)           
    
    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(image, 0.8, line_image, 1, 0)
    
    return lines_edges


In [21]:
####code for test videos
#if os.path.isdir("test_videos_output"):
#    pass
#else:
#    os.mkdir("test_videos_output")
white_output = "C:/Users/dell/test_videos_output/solidYellowLeft.mp4"
clip1 = VideoFileClip("C:/Users/dell/CarND-LaneLines-P1/test_videos/solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
clip1.reader.close()

###code for test images

img_list = os.listdir('C:/Users/dell/CarND-LaneLines-P1/test_images/')
if os.path.isdir("test_images_output"):
    pass
else:
    os.mkdir("test_images_output")
    
#image = mpimg.imread('C:/Users/dell/CarND-LaneLines-P1/test_images/whiteCarLaneSwitch.jpg')
#image_output = process_image(image)
#plt.imshow(image_output,cmap='gray')
for img in img_list:
    image = mpimg.imread('C:/Users/dell/CarND-LaneLines-P1/test_images/'+img)
    image_output = process_image(image)
    mpimg.imsave(("test_images_output/"+img),image_output,cmap='gray')